In [172]:
import requests
import json
import pandas as pd
import collections

In [66]:
r = requests.get('http://dataservices.imf.org/RESsT/SDMX_JSON.svc/Dataflow/')

In [45]:
def DataflowMethod():
    r = requests.get('http://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow/')
    IDs = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['KeyFamilyRef'].apply(lambda x: x['KeyFamilyID'])
    Texts = pd.DataFrame(r.json()['Structure']['Dataflows']['Dataflow'])['Name'].apply(lambda x: x['#text'])
    return(pd.DataFrame({'DatabaseID': IDs, 
                         'DatabaseText': Texts}))

In [170]:
def DataStructureMethod(database_id, check_query = False):
    
    if check_query:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None

    r = requests.get(''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/',database_id]))
    if not r.ok:
        return None

    rparsed = r.json()
    dim_code = pd.Series(rparsed['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']).apply(lambda x: x['@codelist']).tolist()
    dim_codedict = [x for x in rparsed['Structure']['CodeLists']['CodeList'] if x['@id'] in dim_code]
    dim_codedict = dict(zip(pd.Series(dim_codedict).apply(lambda x: x['@id']).tolist(), dim_codedict))
    for k in dim_codedict.keys():
        dim_codedict[k] = pd.DataFrame({
                'CodeValue' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['@value']),
                'CodeText' : pd.Series(dim_codedict[k]['Code']).apply(lambda x: x['Description']['#text'])
            })
    
    return((dim_code, dim_codedict))

In [201]:
def CompactDataMethod(database_, queryfilter=None, 
                      startdate='2001-01-01', enddate='2001-12-31',
                      checkquery = False, verbose=False, tidy=False):
    
    if checkquery:
        available_datasets = DataflowMethod()['DatabaseID'].tolist()
        if not database_id in available_datasets:
            return None
        
    queryfilterstr = ''
    if len(queryfilter[1]) > 0 and len(queryfilter[0]) == len(queryfilter[1]):
        queryfilterstr = '.'.join(['+'.join(queryfilter[1][k]) for k in queryfilter[0]])
        
    APIstr = ''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/',
                      databaseID,'/',queryfilterstr,'?startPeriod=',startdate,'&endPeriod=',enddate])
    r = requests.get(APIstr)        
    
    if verbose:
        print('\nmaking API call:\n')
        print(APIstr)
        print('\n')
        
    if not r.ok:
        return None        

    rparsed = r.json()
    
    if(is.null(r.parsed$CompactData$DataSet$Series)){
    warning("No data available")
    return(NULL)
    }

    if(class(r.parsed$CompactData$DataSet$Series) == "data.frame"){
    r.parsed$CompactData$DataSet$Series <- r.parsed$CompactData$DataSet$Series[!plyr::laply(r.parsed$CompactData$DataSet$Series$Obs, is.null),]
    if(nrow(r.parsed$CompactData$DataSet$Series) ==0){
      warning("No data available")
      return(NULL)
    }
    }

    if(class(r.parsed$CompactData$DataSet$Series) == "list"){
    if(is.null(r.parsed$CompactData$DataSet$Series$Obs)){
      warning("No data available")
      return(NULL)
    }
    ret.df <- as.data.frame(r.parsed$CompactData$DataSet$Series[1:(length(r.parsed$CompactData$DataSet$Series)-1)])
    ret.df$Obs <- list(r.parsed$CompactData$DataSet$Series$Obs)
    names(ret.df) <- names(r.parsed$CompactData$DataSet$Series)
    r.parsed$CompactData$DataSet$Series <- ret.df
    }

    if(tidy){
    ret.df <- r.parsed$CompactData$DataSet$Series
    for(i in 1:length(ret.df$Obs)){
      ret.df$Obs[[i]] <- merge(ret.df$Obs[[i]], ret.df[i,1:(ncol(ret.df)-1)])
    }
    ret.df <- plyr::ldply(ret.df$Obs)
    return(ret.df)
    }


In [ ]:
queryfilter = (['CL_FREA','CL_AREA_IFS','CL_INDICATOR_IFS'], 
               {'CL_FREA':['Q','A'],
                'CL_AREA_IFS':['GR'],
                'CL_INDICATOR_IFS':['NGDP_EUR','NGDP_XDC']})

startdate = '2001-01-01'
enddate='2001-12-31'
checkquery = False
verbose=False
tidy=False

queryfilterstr = ''
if len(queryfilter[1]) > 0 and len(queryfilter[0]) == len(queryfilter[1]):
    queryfilterstr = '.'.join(['+'.join(queryfilter[1][k]) for k in queryfilter[0]])

APIstr = ''.join(['http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/',
                  databaseID,'/',queryfilterstr,'?startPeriod=',startdate,'&endPeriod=',enddate])

In [205]:
teststr = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q+A.GR.NGDP_EUR+NGDP_XDC?startPeriod=2001-01-01&endPeriod=2001-12-31'

In [206]:
r = requests.get(teststr)        
rparsed = r.json()

In [212]:
rparsed['CompactData']['DataSet']['Series']

[{'@FREQ': 'Q',
  '@INDICATOR': 'NGDP_EUR',
  '@REF_AREA': 'GR',
  '@TIME_FORMAT': 'P3M',
  '@UNIT_MULT': '9',
  'Obs': [{'@OBS_STATUS': 'K',
    '@OBS_VALUE': '35.2366',
    '@TIME_PERIOD': '2001-Q1'},
   {'@OBS_VALUE': '36.7264', '@TIME_PERIOD': '2001-Q2'},
   {'@OBS_VALUE': '39.8428', '@TIME_PERIOD': '2001-Q3'},
   {'@OBS_VALUE': '40.3881', '@TIME_PERIOD': '2001-Q4'}]},
 {'@FREQ': 'Q',
  '@INDICATOR': 'NGDP_XDC',
  '@REF_AREA': 'GR',
  '@TIME_FORMAT': 'P3M',
  '@UNIT_MULT': '9'},
 {'@FREQ': 'A',
  '@INDICATOR': 'NGDP_XDC',
  '@REF_AREA': 'GR',
  '@TIME_FORMAT': 'P1Y',
  '@UNIT_MULT': '9'},
 {'@FREQ': 'A',
  '@INDICATOR': 'NGDP_EUR',
  '@REF_AREA': 'GR',
  '@TIME_FORMAT': 'P1Y',
  '@UNIT_MULT': '9',
  'Obs': {'@OBS_STATUS': 'K',
   '@OBS_VALUE': '151.9872',
   '@TIME_PERIOD': '2001'}}]

In [ ]:
CompactDataMethod <- function(databaseID, queryfilter,
                              startdate='2001-01-01', enddate='2001-12-31',
                              checkquery = FALSE, verbose=FALSE, tidy=FALSE){
  if(checkquery){
    available.datasets <- DataflowMethod()$DatabaseID
    if (!is.element(databaseID, available.datasets)){
      stop('databaseID is not exist in API')
      return(NULL)
    }
    acceptedquery <- DataStructureMethod(databaseID)
    if (length(queryfilter) !=0 ||
        length(queryfilter) != length(acceptedquery)){
      stop('queryfilter is wrong format')
      return(NULL)
    }
  }

  queryfilterstr <- ''
  if (length(queryfilter) > 0){
    queryfilterstr <- paste0(
      unlist(plyr::llply(queryfilter,
                         function(x)(paste0(x, collapse="+")))), collapse=".")
  }

  APIstr <- paste0('http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/',
                    databaseID,'/',queryfilterstr,
                    '?startPeriod=',startdate,'&endPeriod=',enddate)
  r <- httr::GET(APIstr)

  if(verbose){
    cat('\nmaking API call:\n')
    cat(APIstr)
    cat('\n')
  }

  if(httr::http_status(r)$reason != "OK"){
    stop(paste(unlist(httr::http_status(r))))
    return(list())
  }
  r.parsed <- jsonlite::fromJSON(httr::content(r, "text"))

  if(is.null(r.parsed$CompactData$DataSet$Series)){
    warning("No data available")
    return(NULL)
  }

  if(class(r.parsed$CompactData$DataSet$Series) == "data.frame"){
    r.parsed$CompactData$DataSet$Series <- r.parsed$CompactData$DataSet$Series[!plyr::laply(r.parsed$CompactData$DataSet$Series$Obs, is.null),]
    if(nrow(r.parsed$CompactData$DataSet$Series) ==0){
      warning("No data available")
      return(NULL)
    }
  }

  if(class(r.parsed$CompactData$DataSet$Series) == "list"){
    if(is.null(r.parsed$CompactData$DataSet$Series$Obs)){
      warning("No data available")
      return(NULL)
    }
    ret.df <- as.data.frame(r.parsed$CompactData$DataSet$Series[1:(length(r.parsed$CompactData$DataSet$Series)-1)])
    ret.df$Obs <- list(r.parsed$CompactData$DataSet$Series$Obs)
    names(ret.df) <- names(r.parsed$CompactData$DataSet$Series)
    r.parsed$CompactData$DataSet$Series <- ret.df
  }

  if(tidy){
    ret.df <- r.parsed$CompactData$DataSet$Series
    for(i in 1:length(ret.df$Obs)){
      ret.df$Obs[[i]] <- merge(ret.df$Obs[[i]], ret.df[i,1:(ncol(ret.df)-1)])
    }
    ret.df <- plyr::ldply(ret.df$Obs)
    return(ret.df)
  }

  return(r.parsed$CompactData$DataSet$Series)
}

In [171]:
# DataflowMethod()
DataStructureMethod('IFS')

(['CL_FREQ', 'CL_AREA_IFS', 'CL_INDICATOR_IFS'],
 {'CL_AREA_IFS':                                               CodeText  \
  0                                          Afghanistan   
  1                                              Albania   
  2                                              Algeria   
  3                                       American Samoa   
  4                                               Angola   
  5                                             Anguilla   
  6                                  Antigua and Barbuda   
  7                             AMF (Arab Monetary Fund)   
  8                                            Argentina   
  9                                              Armenia   
  10                                               Aruba   
  11                                           Australia   
  12                                             Austria   
  13                                          Azerbaijan   
  14                                

In [161]:
list(adict)

['c', 'a', 'b']

In [167]:
[x for x in adict.values()]

[3, 1, 2]

In [169]:
adict.keys()

dict_keys(['c', 'a', 'b'])

In [174]:
adict = collections.OrderedDict({'a':1, 'b':2, 'c':3})